In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.datasets import mnist

In [ ]:
target_shape = (28, 28, 1)
target_size = (28, 28)

epochs = 100
batch_size = 32
latent_variables = 100

In [ ]:
#Function to load the train_data

def get_data():
    (X, _), (_, _) = mnist.load_data()
    X = X.astype(np.float32)/255.
    X = np.expand_dims(X, -1)
    return X

In [ ]:
#Function to show a data sample. 
def show_data_sample(imgs, rows):
    
    fig, ax = plt.subplots(rows, rows, constrained_layout = True, figsize = (10, 10))
    for i in range(rows):
        for j in range(rows):
            ax[i][j].imshow(imgs[j + 5*i,:,:,0], cmap = 'binary')
            ax[i][j].axis('off')
    plt.show()

In [ ]:
#Generator Model

def generator_module(shape = latent_variables):
    
    model = models.Sequential()
    model.add(layers.Dense(7*7*128, input_shape = [latent_variables]))
    model.add(layers.LeakyReLU(alpha = 0.2))
    
    #Reshaping Layer. This is consequently upsampled to the target shape requirement
    model.add(layers.Reshape((7, 7, 128)))
    
    model.add(layers.Conv2DTranspose(64, (3, 3), strides = (2, 2), padding = 'same'))
    model.add(layers.LeakyReLU(alpha = 0.2))
    
    model.add(layers.Conv2DTranspose(32, (5, 5), strides = (2, 2), padding = 'same'))
    model.add(layers.LeakyReLU(alpha = 0.2))
    
    model.add(layers.Conv2DTranspose(1, (5, 5), strides = (1, 1), padding = 'same'))
    model.add(layers.Activation('tanh'))
    
    #Tanh and Leaky Relu activations help prevent Mode Collapse
    model.summary()
    return model

In [ ]:
#Discriminator Model

def discriminator_module(shape = target_shape):
    
    model = models.Sequential()
    
    #Downsampling is done using Strided Convolutions, so that 
    #the model learns its own spatial downsampling
    
    model.add(layers.Conv2D(64, (3,3), strides = (2, 2), padding = 'same', input_shape = shape))
    model.add(layers.LeakyReLU(alpha = 0.2))
    
    model.add(layers.Conv2D(128, (3, 3), strides = (2, 2), padding = 'same'))
    model.add(layers.LeakyReLU(alpha = 0.2))
    
    model.add(layers.Conv2D(256, (3, 3), strides = (2, 2), padding = 'same'))
    model.add(layers.LeakyReLU(alpha = 0.2))
    
    #Final Dense Layers for classification
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.4))                  #Dropout for regularization
    model.add(layers.Dense(1, activation = 'sigmoid'))
    
    model.summary()
    return model

In [ ]:
#Generates the required number random noise vectors of a fixed length
def get_random_space(num_features, num_samples):
    
    random_space = np.random.randn(num_features*num_samples)
    random_space = random_space.reshape((num_samples, num_features))
    return random_space

In [ ]:
#Returns the required number of images from random index points from the source dataset
def get_real_images(source_dataset, num_samples):
    
    random_idx = np.random.randint(0, source_dataset.shape[0], num_samples)
    real_data = source_dataset[random_idx]
    return real_data

In [ ]:
#Returns the required number of generated images 
def get_fake_images(gen, num_features, num_samples):
    
    gen_input = get_random_space(num_features, num_samples)        #Random Noise vectors for generator
    fake_data = gen.predict(gen_input)            #Generated Images
    return fake_data

In [ ]:
#Function to plot and save result after each epoch 
def save_output(gen, epoch, seed):
    imgs = gen.predict(seed)
    
    fig, ax = plt.subplots(5, 5, constrained_layout = True, figsize = (10, 10))
    for i in range(5):
        for j in range(5):
            ax[i][j].imshow(imgs[j + 5*i,:,:,0], cmap = 'binary')
            ax[i][j].axis('off')
    plt.savefig('GenImg{:04d}'.format(epoch))
    plt.show()

In [ ]:
#Function to train the DCGAN
def train_dcgan(gan, data, num_features = latent_variables, batch_size = batch_size, epochs = epochs):
    
    #Generator and Discriminator are attatched sequentially in the GAN, they can be extracted as follows
    gen, disc = gan.layers
    num_batches = int(data.shape[0]/batch_size)
    
    for epoch in range(epochs):
        
        for i in tqdm(range(num_batches), ascii = True, desc = 'Epoch {}/{}'.format(epoch + 1, epochs), ncols = 100):
            
            #Initially, the discriminator is frozen, so we unfreeze it for training
            disc.trainable = True
            
            #Real Data samples are generated using the defined functions
            #Labels are assigned as 1 for real images 
            X_real = get_real_images(data, batch_size)
            y_real = np.ones((batch_size, 1))
            
            disc_loss_real, _ = disc.train_on_batch(X_real, y_real)
            
            #Fake Data samples are generated using the defined functions
            #Labels are assigned as 0 for fake images
            X_fake = get_fake_images(gen, num_features, batch_size)
            y_fake = np.zeros((batch_size, 1))
            
            disc_loss_fake, _ = disc.train_on_batch(X_fake, y_fake)
            
            #To Train the generator, we freeze the discriminator and train the whole GAN
            disc.trainable = False
            
            #Random noise is given as input to GAN
            #As the Generator works opposite to the Discriminator, we invert the labels
            #Fake samples are labeled as 1
            X_gan = get_random_space(num_features, batch_size)
            y_gan = np.ones((batch_size, 1))
            
            #GAN is trained on the created batch
            gan_loss, gan_acc = gan.train_on_batch(X_gan, y_gan)
            
        seed = get_random_space(num_features, batch_size)
        save_output(gen, epoch + 1, seed)
        

In [ ]:
data = get_data()
show_data_sample(data[:25], 5)

In [ ]:
#Creating the GAN architecture
gen = generator_module()

disc = discriminator_module()
disc.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
disc.trainable = False

gan = models.Sequential([gen, disc])
gan.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
#Training the GAN
gan_trained = train_dcgan(gan, data, num_features = latent_variables, batch_size = batch_size, epochs = epochs)